In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv('train.csv')[['Age','Fare','SibSp','Parch','Survived']]

In [3]:
df.dropna(inplace=True)

In [4]:
df['family'] = df['SibSp'] + df['Parch']

In [5]:
df.drop(columns=['SibSp','Parch'],inplace=True)

In [6]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
X_train.head()

,Age,Fare,family
328,31.0,20.5250,2
73,26.0,14.4542,1
253,30.0,16.1000,1
719,33.0,7.7750,0
666,25.0,13.0000,0


## Uniform Bining (Equal Width)

In [9]:
uniform = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')

In [11]:
uni_trf = ColumnTransformer([
    ('first',uniform,[0]),
    ('second',uniform,[1])
])

In [13]:
X_train_uni_trf = uni_trf.fit_transform(X_train)
X_test_uni_trf = uni_trf.transform(X_test)

In [14]:
uni_clf = DecisionTreeClassifier()
uni_clf.fit(X_train_uni_trf,y_train)
uni_y_pred = uni_clf.predict(X_test_uni_trf)
accuracy_score(y_test,uni_y_pred)

0.6783216783216783

## Quantile Bining (Equal Frequency)

In [15]:
quantile = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')

In [16]:
quan_trf = ColumnTransformer([
    ('first',quantile,[0]),
    ('second',quantile,[1])
])

In [17]:
X_train_quan_trf = quan_trf.fit_transform(X_train)
X_test_quan_trf = quan_trf.transform(X_test)

In [18]:
quan_clf = DecisionTreeClassifier()
quan_clf.fit(X_train_quan_trf,y_train)
quan_y_pred = quan_clf.predict(X_test_quan_trf)
accuracy_score(y_test,quan_y_pred)

0.6223776223776224

## KMeans Bining

In [19]:
kmeans = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='kmeans')

In [20]:
kmen_trf = ColumnTransformer([
    ('first',kmeans,[0]),
    ('second',kmeans,[1])
])

In [21]:
X_train_kmen_trf = kmen_trf.fit_transform(X_train)
X_test_kmen_trf = kmen_trf.transform(X_test)

In [22]:
kmen_clf = DecisionTreeClassifier()
kmen_clf.fit(X_train_kmen_trf,y_train)
kmen_y_pred = kmen_clf.predict(X_test_kmen_trf)
accuracy_score(y_test,kmen_y_pred)

0.6083916083916084

## Custom Bining

In [23]:
def discretize(bins,strategy):
    kbin_age = KBinsDiscretizer(n_bins=bins,encode='ordinal',strategy=strategy)
    kbin_fare = KBinsDiscretizer(n_bins=bins,encode='ordinal',strategy=strategy)
    
    trf = ColumnTransformer([
        ('first',kbin_age,[0]),
        ('second',kbin_fare,[1])
    ])
    
    X_trf = trf.fit_transform(X)
    print(np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy')))
    
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    plt.hist(X['Age'])
    plt.title("Before")

    plt.subplot(122)
    plt.hist(X_trf[:,0],color='red')
    plt.title("After")

    plt.show()
    
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    plt.hist(X['Fare'])
    plt.title("Before")

    plt.subplot(122)
    plt.hist(X_trf[:,1],color='red')
    plt.title("Fare")

    plt.show()
    